# Le seguenti 3 celle fanno il calcolo della distanza di una coordinata sul file SPT-CL_J2344-4243_band3_combined.dat.im.image.fits

La prima cella fa un import dei package necessari. La seconda definisce una funzione con tutte le eccezioni del caso e restituisce 3 distanze: [distanza in pixel, distanza in gradi, distanza in radianti]. 

Gli argomenti della funzione sono: il file fits su cui basarsi, le coordinate x e y dei due punti (per il secondo punto ci sono dei valori di default che sono quelli del reference point nell'header del file) di cui calcolare la distanza, l'unità di misura con cui sono espresse queste coordinate. Quest'ultima può essere in pixel ('pixel'), gradi sessagesimali ('angle_deg') o radianti ('angle_rad').

In [2]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def distance_calc(fil, x_1 , y_1 , x_0=-9999999.9 , y_0=-9999999.9 , scale='pixel'):
    if scale in ['pixel' , 'angle_rad' , 'angle_deg']:
        
        sci=fits.open(fil)[0]
        hdr=sci.header
        dat=sci.data
        if scale=='pixel':
            if (x_0 , y_0)==(-9999999.9 , -9999999.9):
                x_0 = hdr['CRPIX1']
                y_0 = hdr['CRPIX2']
        
            if x_1 in np.arange (0, float(hdr['NAXIS1'])) and y_1 in np.arange (0, float(hdr['NAXIS2'])):
                
                    
                if x_0 in np.arange (0, float(hdr['NAXIS1'])) and y_0 in np.arange (0, float(hdr['NAXIS2'])):
                    
                        
                    dist_x = float(x_1) - float(x_0)
                    dist_y = float(y_1) - float(y_0)
                    dist_pix = np.sqrt((1.0*dist_x)**2 + (1.0*dist_y)**2)
                    dist_deg = np.sqrt((float(hdr['CDELT1'])*dist_x)**2 + (float(hdr['CDELT2'])*dist_y)**2)
                    dist_rad = dist_deg*np.pi/180.0
                    dist = [dist_pix , dist_deg , dist_rad]
                else:
                    print("Centre coordinates are out of range. ")
                    dist=[np.nan , np.nan , np.nan]
            else:
                print("Given point's coordinates are out of range. ")
                dist==[np.nan , np.nan , np.nan]
        elif scale=='angle_deg':
            if (x_0 , y_0)==(-9999999.9 , -9999999.9):
                x_0 = hdr['CRVAL1']
                x_0 = hdr['CRVAL2']
                
            if x_1 in np.arange (float(hdr['CRVAL1']) - float(hdr['CRPIX1'])*float(hdr['CDELT1']) , float(hdr['CRVAL1']) + (float(hdr['NAXIS1']) - float(hdr['CRPIX1']))*float(hdr['CDELT1'])) and y_1 in np.arange (float(hdr['CRVAL2']) - float(hdr['CRPIX2'])*float(hdr['CDELT2']) , float(hdr['CRVAL2']) + (float(hdr['NAXIS2']) - float(hdr['CRPIX2']))*float(hdr['CDELT2'])):                
                if x_0 in np.arange (float(hdr['CRVAL1']) - float(hdr['CRPIX1'])*float(hdr['CDELT1']) , float(hdr['CRVAL1']) + (float(hdr['NAXIS1']) - float(hdr['CRPIX1']))*float(hdr['CDELT1'])) and y_0 in np.arange (float(hdr['CRVAL2']) - float(hdr['CRPIX2'])*float(hdr['CDELT2']) , float(hdr['CRVAL2']) + (float(hdr['NAXIS2']) - float(hdr['CRPIX2']))*float(hdr['CDELT2'])):
                    dist_x = float(x_1) - float(x_0)
                    dist_y = float(y_1) - float(y_0)
                    dist_deg = np.sqrt((1.0*dist_x)**2 + (1.0*dist_y)**2)
                    dist_pix = np.sqrt((dist_x/float(hdr['CDELT1']))**2 + (dist_y/float(hdr['CDELT2']))**2)
                    dist_rad = dist_deg*np.pi/180.0
                    dist = [dist_pix , dist_deg , dist_rad]
                else:
                    print("Centre coordinates are out of range. ")
                    dist=[np.nan , np.nan , np.nan]
            else:
                print("Given point's coordinates are aout of range. ")
                dist==[np.nan , np.nan , np.nan]     
        elif scale=='angle_rad':
            if (x_0 , y_0)==(-9999999.9 , -9999999.9):
                x_0 = hdr['CRVAL1']*np.pi/180.0
                x_0 = hdr['CRVAL2']*np.pi/180.0
                
            if x_1 in range (float(hdr['CRVAL1'])*np.pi/180.0 - float(hdr['CRPIX1'])*float(hdr['CDELT1'])*np.pi/180.0 , float(hdr['CRVAL1'])*np.pi/180.0 + (float(hdr['NAXIS1']) - float(hdr['CRPIX1']))*float(hdr['CDELT1'])*np.pi/180.0) and y_1 in np.arange (float(hdr['CRVAL2'])*np.pi/180.0 - float(hdr['CRPIX2'])*float(hdr['CDELT2'])*np.pi/180.0 , float(hdr['CRVAL2'])*np.pi/180.0 + (float(hdr['NAXIS2']) - float(hdr['CRPIX2']))*float(hdr['CDELT2'])*np.pi/180.0):                
                if x_0 in range (float(hdr['CRVAL1'])*np.pi/180.0 - float(hdr['CRPIX1'])*float(hdr['CDELT1'])*np.pi/180.0 , float(hdr['CRVAL1'])*np.pi/180.0 + (float(hdr['NAXIS1']) - float(hdr['CRPIX1']))*float(hdr['CDELT1'])*np.pi/180.0) and y_0 in np.arange (float(hdr['CRVAL2'])*np.pi/180.0 - float(hdr['CRPIX2'])*float(hdr['CDELT2'])*np.pi/180.0 , float(hdr['CRVAL2'])*np.pi/180.0 + (float(hdr['NAXIS2']) - float(hdr['CRPIX2']))*float(hdr['CDELT2'])*np.pi/180.0):               
                    dist_x = float(x_1) - float(x_0)
                    dist_y = float(y_1) - float(y_0)
                    dist_rad = np.sqrt((1.0*dist_x)**2 + (1.0*dist_y)**2)
                    dist_pix = np.sqrt((dist_x/float(hdr['CDELT1']))**2 + (dist_y/float(hdr['CDELT2']))**2)*180.0/np.pi
                    dist_deg = dist_rad*180.0/np.pi
                    dist = [dist_pix , dist_deg , dist_rad]    
                else:
                    print("Centre coordinates are out of range. ")
                    dist=[np.nan , np.nan , np.nan]
            else:
                print("Given point's coordinates are aout of range. ")
                dist==[np.nan , np.nan , np.nan]   
                    
                    
                    
                
               
    else:
        print('Can not work with those units. ')
        print('**'*20)
        print('Scale can be "pixel" (default), "angle_deg" or "angle_rad". ')
        dist=np.nan
    return dist

La cella seguente chiama la function facendole calcolare una distanza fra la coordinata (1000.0, 1000.0) e la coordinata di reference espressa nell'header del file

In [7]:
distance_calc('SPT-CL_J2344-4243_band3_combined.dat.im.image.fits' , 1000.0 , 1000.0)

[35.35533905932738, 0.0014731391274720917, 2.5711128114344452e-05]